In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
pd.set_option('display.max_rows', 10)
train.set_index(['user_id'], inplace=True)
test.set_index(['user_id'], inplace=True)

In [76]:
#train.drop(['EmployeeCount'], axis=1)
#test.drop(['EmployeeCount'], axis=1)

In [12]:
lenc = LabelEncoder()
for col in train.columns:
    if train[col].dtype == 'object':
        l = lenc.fit_transform(train[col])
        train[col] = l
for col in test.columns:
    if test[col].dtype == 'object':
        l = lenc.fit_transform(test[col])
        test[col] = l

In [13]:
train_x = train.drop(['Attrition'], axis=1)
train_y = train['Attrition']

In [14]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(train_x, train_y)
predict_y = lr.predict(test)

test_y = test.drop(test.columns[0:34], axis=1, inplace=False)
test_y.insert(0, 'Attrition', predict_y)
test_y = test_y.reset_index()
test_y.to_csv('./lr.csv', index=False)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
# Decision Tree
'''clf = DecisionTreeClassifier()
clf.fit(train_x, train_y)
predict_y = clf.predict(test)

test_y = test.drop(test.columns[0:34], axis=1, inplace=False)
test_y.insert(0, 'Attrition', predict_y)
test_y = test_y.reset_index()
test_y.to_csv('./clf.csv', index=False)'''